In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import xgboost
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder(sparse = False)
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.utils import resample
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, StratifiedKFold
from scipy.stats import uniform, randint

In [ ]:
df = pd.read_csv('Train_Data (2).csv')
df_test = pd.read_csv("Test_Data (3).csv")

In [ ]:
df.duplicated().sum()

0

##Data Cleaning Training Data

In [ ]:
df['Healthy'].value_counts()

1    16300
0     9620
Name: Healthy, dtype: int64

In [ ]:
# rv = df[(df['Social interaction']==0) & (df['Healthy']==0)]
# rv

In [ ]:
# df['Taking supplements'].value_counts()

In [ ]:
# df.describe()

In [ ]:
# sns.displot(df["BMI"], kde=True)
# plt.show()

In [ ]:
# bmi = df[df['BMI']<15]
# bmi

In [ ]:
# # Null value transformation
df['Food preference'] = df['Food preference'].fillna('DX6')
df['Smoker?'] = df['Smoker?'].fillna('NO')
df['Follow Diet'] = df['Follow Diet'].fillna(value = 1)
df['Physical activity']= df['Physical activity'].fillna(value=0)
df['Regular sleeping hours'] = df['Regular sleeping hours'].fillna(value = 0)
df['Alcohol consumption'] =df['Alcohol consumption'].fillna(value=0)
df['Taking supplements'] = df['Taking supplements'].fillna(value=0)
df['Illness count last year'] = df['Illness count last year'].fillna(value=2)
df['Social interaction'] =df['Social interaction'].fillna(value=0)
# df = df.dropna()

In [ ]:
# bmi2 = df[df['BMI']>30]
# bmi2.shape

In [ ]:
# age_f= df[(df['Age']<50) & (df['Healthy']==0)]
# age_f

In [ ]:
# df['Food preference'].value_counts()

In [ ]:
df.head()

,ID1,Specific ailments,ID2,Food preference,Age,BMI,Smoker?,Living in?,Any heriditary condition?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,Taking supplements,Mental health management,Illness count last year,Healthy
0,2408,44,2668,DX6,49,20.500470,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
1,25063,39,10363,DX3 DX4,20,26.076580,NO,URBAN,Stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1
2,26798,29,132,DX6,1,21.420866,NO,URBAN,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,31907,27,10499,DX1,30,25.203247,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1
4,26412,9,7963,DX6,40,19.355846,YES,RURAL,Stable,1.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1


In [ ]:
df_encoded = pd.get_dummies(df['Food preference'].str.split(' ', expand=True).stack()).groupby(level=0).sum()
df = pd.concat([df.drop(columns='Food preference'), df_encoded], axis=1)

In [ ]:
df_encoded2 = pd.get_dummies(df['Illness count last year'])
df_encoded2 = df_encoded2.reset_index(drop = True)
df = pd.concat([df.drop(columns='Illness count last year'), df_encoded2], axis=1)
df.columns = df.columns.astype(str)

In [ ]:
col_list = ['Smoker?','Living in?','Any heriditary condition?','Follow Diet','Physical activity','Regular sleeping hours','Alcohol consumption','Social interaction','Taking supplements','Illness count last year','Healthy']

In [ ]:
# for col in col_list:
#   print(df[col].value_counts())
#   print("\n")

In [ ]:
# for col in col_list:
#   print(df.groupby(col)['Healthy'].value_counts())
#   print("\n")


In [ ]:
# age_70 = df[(df['Age']>70) & (df['Healthy'] ==1)]
# print(age_70.shape)
# age_70
# dropping these columns as people of age greater than 100 with such low BMI cannot be Healthy
df1 = df[df['Age']<=70]

In [ ]:
# df1['Mental health management'].value_counts()

In [ ]:
#dropping heriditary column because every row have same value
df1 = df1.drop(['Any heriditary condition?','ID1','ID2','Mental health management'], axis=1)
df1.head(5)

,Specific ailments,Age,BMI,Smoker?,Living in?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,...,DX2,DX3,DX4,DX5,DX6,1.0,2.0,3.0,4.0,5.0
0,44,49,20.500470,NO,RURAL,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
1,39,20,26.076580,NO,URBAN,0.0,0.0,0.0,0.0,1.0,...,0,1,1,0,0,1,0,0,0,0
2,29,1,21.420866,NO,URBAN,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
3,27,30,25.203247,NO,RURAL,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,9,40,19.355846,YES,RURAL,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
# labeling_list = ['Living in?']
# for col in labeling_list:
#   # df1[col] = le.fit_transform(df1[col])
#   df_encoded = pd.get_dummies(df1[col], prefix=col )
#   df1 = pd.concat([df1, df_encoded], axis=1)
#   df1 = df1.drop(col,axis=1)

In [ ]:
df1['Living in?'] = le.fit_transform(df1['Living in?'])

In [ ]:
df1['Smoker?'] = le.fit_transform(df1['Smoker?'])

In [ ]:
df1.head(5)

,Specific ailments,Age,BMI,Smoker?,Living in?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,...,DX2,DX3,DX4,DX5,DX6,1.0,2.0,3.0,4.0,5.0
0,44,49,20.500470,0,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
1,39,20,26.076580,0,1,0.0,0.0,0.0,0.0,1.0,...,0,1,1,0,0,1,0,0,0,0
2,29,1,21.420866,0,1,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
3,27,30,25.203247,0,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,9,40,19.355846,1,0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
# #adding score column
# # l = df1[(df1['Healthy']==1) & (df1['DX6']==1)]
# # l
# df1.describe()

In [ ]:
col_no = 12
df1 = df1.drop(df1.columns[col_no], axis=1)

In [ ]:
df1.head()

,Specific ailments,Age,BMI,Smoker?,Living in?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,...,DX2,DX3,DX4,DX5,DX6,1.0,2.0,3.0,4.0,5.0
0,44,49,20.500470,0,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
1,39,20,26.076580,0,1,0.0,0.0,0.0,0.0,1.0,...,0,1,1,0,0,1,0,0,0,0
2,29,1,21.420866,0,1,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
3,27,30,25.203247,0,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0
4,9,40,19.355846,1,0,1.0,0.0,0.0,0.0,1.0,...,0,0,0,0,1,0,1,0,0,0


In [ ]:
# df1['pos_score'] = df1['Follow Diet']+ df1['Physical activity'] + df1['Regular sleeping hours']  + df1['Social interaction'] + df1['Taking supplements']
# df1['neg_score'] = - df1['Alcohol consumption'] - df1['Smoker?']

In [ ]:
# df1['neg_score'].value_counts()

In [ ]:
# sns.displot(df1['Age'])

In [ ]:
def resample_data(df1):
    df1_minority = df1[df1['Healthy'] ==0]
    df1_majority = df1[df1['Healthy'] ==1]
    df1_minority = resample(df1_minority, replace = True, n_samples = len(df1_majority), random_state = 0)
    return pd.concat([df1_minority, df1_majority])
df1_res = resample_data(df1)

In [ ]:
y = df1['Healthy']
x = df1.drop('Healthy', axis=1)
x_train, x_test,y_train ,y_test = train_test_split(x, y, random_state = 0, test_size =0.2)

In [ ]:
x_train = scaler.fit_transform(x_train)
x_train = pd.DataFrame(x_train)
x_test = scaler.transform(x_test)
x_test = pd.DataFrame(x_test)

In [ ]:
# y_data_res = df1_res['Healthy']
# x_data_res = df1_res.drop('Healthy',axis=1)

# x_train_res, x_test_res, y_train_res, y_test_res = train_test_split(x_data_res, y_data_res, test_size = 0.2, random_state = 0)

# x_train_res = pd.DataFrame(scaler.fit_transform(x_train_res))
# x_test_res = pd.DataFrame(scaler.transform(x_test_res))


In [ ]:
# x_train.head(1)

##Data Cleaning Testing Data

In [ ]:
df_test

,ID1,Specific ailments,ID2,Food preference,Age,BMI,Smoker?,Living in?,Any heriditary condition?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,Taking supplements,Mental health management,Illness count last year
0,28534,2,3306,DX6,38,18.879331,YES,URBAN,Stable,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
1,7970,5,5573,DX1,46,21.231991,NO,URBAN,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,22039,37,9305,DX4,11,17.867876,NO,RURAL,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,12332,44,8274,DX5,10,26.886096,NO,RURAL,Stable,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0
4,4116,37,4558,DX3 DX4,3,23.362746,NO,RURAL,Stable,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,23747,21,3612,DX3,27,28.880884,NO,URBAN,Stable,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
6476,26138,27,4169,DX3,8,20.358229,NO,RURAL,Stable,1.0,1.0,0.0,0.0,1.0,0.0,0.0,3.0
6477,7536,39,8525,DX6,29,27.365294,NO,RURAL,Stable,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6478,29410,29,1866,DX5,65,22.546412,YES,URBAN,Stable,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:

# sns.displot(df_test["BMI"], kde=True)
# plt.show()

In [ ]:

# bmi = df_test[df_test['BMI']<15]
# bmi

In [ ]:

# df_test['Food preference'].value_counts()

In [ ]:

# bmi2 = df_test[df_test['BMI']>30]
# # bmi2.shape
# df_test['Smoker?'].value_counts()

In [ ]:

# Null value transformation
df_test['Food preference'] = df_test['Food preference'].fillna('DX6')
df_test['Smoker?'] = df_test['Smoker?'].fillna('NO')
df_test['Follow Diet'] = df_test['Follow Diet'].fillna(value = 1)
df_test['Physical activity']= df_test['Physical activity'].fillna(value=0)
df_test['Regular sleeping hours'] = df_test['Regular sleeping hours'].fillna(value = 0)
df_test['Alcohol consumption'] =df_test['Alcohol consumption'].fillna(value=0)
df_test['Taking supplements'] = df_test['Taking supplements'].fillna(value=0)
df_test['Mental health management'] = df_test['Mental health management'].fillna(value=0)
df_test['Illness count last year'] = df_test['Illness count last year'].fillna(value=1)
df_test['Social interaction'] =df_test['Social interaction'].fillna(value=0)

In [ ]:
#encoding
df_test_encoded = pd.get_dummies(df_test['Food preference'].str.split(' ', expand=True).stack()).groupby(level=0).sum()
df_test = pd.concat([df_test.drop(columns='Food preference'), df_test_encoded], axis=1)

df_test_encoded2 = pd.get_dummies(df_test['Illness count last year'])
df_test_encoded2 = df_test_encoded2.reset_index(drop = True)
df_test = pd.concat([df_test.drop(columns='Illness count last year'), df_test_encoded2], axis=1)
df_test.columns = df_test.columns.astype(str)

In [ ]:

# df_test.isnull().sum()

In [ ]:

# col_list = ['Smoker?','Living in?','Any heriditary condition?','Follow Diet','Physical activity','Regular sleeping hours','Alcohol consumption','Social interaction','Taking supplements','Mental health management','Illness count last year']

In [ ]:

# for col in col_list:
#   print(df_test[col].value_counts())
#   print("\n")

In [ ]:

#dropping heriditary column because every row have same value
df1_test = df_test.drop(['Any heriditary condition?','ID1','ID2','Mental health management'], axis=1)
df1_test.head(1)

,Specific ailments,Age,BMI,Smoker?,Living in?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,...,DX2,DX3,DX4,DX5,DX6,1.0,2.0,3.0,4.0,5.0
0,2,38,18.879331,YES,URBAN,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0


In [ ]:

# labeling_list = ['Living in?']
# for col in labeling_list:
#   # df1[col] = le.fit_transform(df1[col])
#   df_encoded = pd.get_dummies(df1_test[col], prefix=col )
#   df1_test = pd.concat([df1_test, df_encoded], axis=1)
#   df1_test = df1_test.drop(col,axis=1)

df1_test['Living in?'] = le.fit_transform(df1_test['Living in?'])

In [ ]:

df1_test['Smoker?'] = df1_test['Smoker?'].replace('Cannot say', 'NO')

In [ ]:
df1_test['Smoker?'].value_counts()

NO     3644
YES    2836
Name: Smoker?, dtype: int64

In [ ]:

#df1_test[df1_test['Smoker?'] == 'Cannot say']
df1_test['Smoker?'] = le.fit_transform(df1_test['Smoker?'])

In [ ]:
col_no = 11
df1_test = df1_test.drop(df1_test.columns[col_no], axis=1)
df1_test.head(2)

,Specific ailments,Age,BMI,Smoker?,Living in?,Follow Diet,Physical activity,Regular sleeping hours,Alcohol consumption,Social interaction,...,DX2,DX3,DX4,DX5,DX6,1.0,2.0,3.0,4.0,5.0
0,2,38,18.879331,1,1,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,1,1,0,0,0,0
1,5,46,21.231991,0,1,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,1,0,0,0,0


In [ ]:
# df1_test['pos_score'] = df1_test['Follow Diet']+ df1_test['Physical activity'] + df1_test['Regular sleeping hours']  + df1_test['Social interaction'] + df1_test['Taking supplements']
# df1_test['neg_score'] = - df1_test['Alcohol consumption'] - df1_test['Smoker?']

In [ ]:
# #@title
# sns.displot(df1_test['Age'])

In [ ]:
df2_test = scaler.transform(df1_test)
df2_test= pd.DataFrame(df2_test)

In [ ]:
# #@title
# df2_test.head()

#Model Training


In [ ]:
def print_stats(df_x,df_y, clf):
  pred = clf.predict(df_x)
  pred = pd.DataFrame({'pred':pred})
  print(f"F1 Score: {f1_score(df_y,pred)}")
  #print(f"accuracy: {accuracy_score(df_y, pred)}")
  # print(classification_report(df_y,pred))
  print(pred.value_counts())

In [ ]:
# y_train.value_counts()

In [ ]:
# y_test.value_counts()

In [ ]:
class_weights = {0 :2, 1:1}
# feature_names = df.columns.tolist()


##LogisticRegression

---



---



In [ ]:
# clf = LogisticRegression()
# clf.fit(x_train, y_train)

# print_stats(x_train,y_train, clf)
# print("\n----------------\n")
# print_stats(x_test, y_test, clf)

In [ ]:
# clf2 = LogisticRegression()
# clf2.fit(x_train_res, y_train_res)

# print_stats(x_train_res,y_train_res, clf2)
# print("\n----------------\n")
# print_stats(x_test_res, y_test_res, clf2)

In [ ]:
# pred = clf.predict(df1_test)
# pred_LR = pd.DataFrame({'predictions':pred})
# pred_LR.value_counts()

In [ ]:
# file_path = 'pred_LR.csv'
# pred_LR.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

## Random Forest

In [ ]:
# clf = RandomForestClassifier(max_depth = 15,n_estimators =100,class_weight= class_weights, random_state =0)
# clf.fit(x_train, y_train)

# print_stats(x_train,y_train, clf)
# print("\n-----------------------\n")
# print_stats(x_test,y_test, clf)


In [ ]:
# clf2 = RandomForestClassifier(n_estimators =500, random_state =0)
# clf2.fit(x_train_res, y_train_res)

# print_stats(x_train_res,y_train_res, clf2)
# print("\n-----------------------\n")
# print_stats(x_test_res,y_test_res, clf2)


In [ ]:
# pred = clf.predict(df2_test)
# pred_RF = pd.DataFrame({'predictions':pred})
# pred_RF.value_counts()

In [ ]:
# file_path = 'pred_RF.csv'
# pred_RF.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

##Gradient Boosting

In [ ]:
# clf = GradientBoostingClassifier()
# clf.fit(x_train,y_train)

# print_stats(x_train, y_train, clf)
# print('\n----------------\n')
# print_stats(x_test, y_test,clf)

In [ ]:
# clf2 = GradientBoostingClassifier()
# clf2.fit(x_train_res, y_train_res)

# print_stats(x_train_res,y_train_res, clf2)
# print("\n----------------\n")
# print_stats(x_test_res, y_test_res, clf2)

In [ ]:
# pred = clf.predict(df1_test)
# pred_gb = pd.DataFrame({'predictions':pred})
# pred_gb.value_counts()

In [ ]:
# file_path = 'pred_gb.csv'
# pred_gb.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

##XGB Classifier

In [ ]:
# clf = xgboost.XGBClassifier()
# clf.fit(x_train, y_train)

# print_stats(x_train, y_train, clf)
# print('\n----------------\n')
# print_stats(x_test, y_test, clf)

In [ ]:
# clf2 = xgboost.XGBClassifier()
# clf2.fit(x_train_res, y_train_res)

# print_stats(x_train_res,y_train_res, clf2)
# print("\n----------------\n")
# print_stats(x_test_res, y_test_res, clf2)

In [ ]:
# #Using Hyperparameter tuning

# param_dist = {
#     'learning_rate': uniform(0.01, 0.1),
#     'max_depth': randint(3, 15),
#     'n_estimators': randint(10, 300),
#     'subsample': uniform(0.6, 0.4),
#     'colsample_bytree': uniform(0.6, 0.4),
# }

# clf = xgboost.XGBClassifier()
# random_search = RandomizedSearchCV(estimator=clf, param_distributions=param_dist, scoring='accuracy', cv=5)
# random_search.fit(x_train, y_train)

# best_params = random_search.best_params_
# best_model = random_search.best_estimator_

# print_stats(x_train, y_train, best_model)
# print('\n----------------\n')
# print_stats(x_test, y_test, best_model)

In [ ]:
# #using gridsearchcv and HP tuning
# param_grid = {
#     'learning_rate': [0.1, 0.01, 0.001],
#     'max_depth': [3, 5, 7],
#     'n_estimators': [100, 200, 300],
#     'subsample': [0.8, 1.0],
#     'colsample_bytree': [0.8, 1.0],
# }
# clf = xgboost.XGBClassifier()
# grid_search = GridSearchCV(estimator=clf, param_grid=param_grid, scoring='accuracy', cv=5)
# grid_search.fit(x_train, y_train)

# best_params = grid_search.best_params_
# best_model = grid_search.best_estimator_

# print_stats(x_train, y_train, best_model)
# print('\n----------------\n')
# print_stats(x_test, y_test, best_model)
# # not sub

In [ ]:
# pred = best_model.predict(df2_test)
# pred_xgb = pd.DataFrame({'predictions':pred})
# pred_xgb.value_counts()

In [ ]:
# file_path = 'pred_xgb_final.csv'
# pred_xgb.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)
# # not sub

##SVM

In [ ]:
# clf = svm.SVC(kernel = 'rbf', C=50, gamma = 'auto')
# clf.fit(x_train, y_train)

# print_stats(x_train, y_train, clf)
# print("\n------------------\n")
# print_stats(x_test, y_test, clf)

In [ ]:
# clf2 = svm.SVC(kernel = 'rbf', C=50, gamma = 'auto',class_weight =class_weights)
# clf2.fit(x_train_res, y_train_res)

# print_stats(x_train_res,y_train_res, clf2)
# print("\n----------------\n")
# print_stats(x_test_res, y_test_res, clf2)

In [ ]:
# pred = clf.predict(df1_test)
# pred_svm = pd.DataFrame({'predictions':pred})
# pred_svm.value_counts()

In [ ]:
# file_path = 'pred_svm.csv'
# pred_svm.to_csv(file_path,index = False)
# from IPython.display import FileLink
# FileLink(file_path)

##ANN

In [ ]:
pip install -U keras-tuner

In [ ]:
from tensorflow import keras
import kerastuner as kt
def print_stats(df_x, df_y, model, threshold=0.5):
    pred_probs = model.predict(df_x)
    pred_labels = np.where(pred_probs > threshold, 1, 0)
    pred = pd.DataFrame({'pred': pred_labels.flatten()})
    f1 = f1_score(df_y, pred)
    print(f"F1 Score: {f1}")
    print(pred['pred'].value_counts())

<ipython-input-85-1a3b2fa0f609>:2: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [ ]:
# #hyperparameter tuning
# def build_model(hp):
#   input_dim = 22
#   model = keras.Sequential([
#     keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
#   ])

#   optimizer = hp.Choice('optimizer', values = ['adam', 'sgd','rmsprop','adadelta'])
#   model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

#   return model

In [ ]:
# tuner = kt.RandomSearch(build_model, objective = 'val_accuracy', max_trials =5 )

In [ ]:
# tuner.search(x_train, y_train, epochs =10, validation_data =(x_test, y_test))

In [ ]:
# tuner.get_best_hyperparameters()[0].values

In [ ]:
# model = tuner.get_best_models(num_models=1)[0]

In [ ]:
# model.summary()

In [ ]:
# model.fit(x_train, y_train, batch_size=32, epochs=100, initial_epoch = 11, validation_data = (x_test, y_test))

In [ ]:
def build_model(hp):
  model = keras.Sequential()

  units = hp.Int('units', min_value=8, max_value=128, step=8)
  input_dim = 22
  model = keras.Sequential([
    keras.layers.Dense(units = units, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(1, activation='sigmoid')
  ])

  model.compile(optimizer='adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
  return model

In [ ]:
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=5 )

In [ ]:
tuner.search(x_train, y_train,epochs=5, validatoin_data =(x_test, y_test))

In [ ]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [ ]:
# input_dim = 22
# model = keras.Sequential([
#     keras.layers.Dense(32, activation='relu', input_shape=(input_dim,)),
#     keras.layers.Dense(64, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')
# ])
# model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(x_train, y_train, epochs=100, batch_size=32)


# print_stats(x_test, y_test, model)
# # print("Test Loss:", loss)
# # print("Test Accuracy:", accuracy)